In [ ]:
import pathlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from matplotlib import rc, rcParams

%matplotlib inline

%load_ext autoreload
%autoreload 2

rc('text', usetex=True)
font = {'family': 'Times New Roman', 'weight': 'bold', 'size': 14}
rc('font', **font)
rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']

In [ ]:
model_vals = []
attention_vals = []
overall_scores_rows = []
columns = None

def iter_results_files(*dir_names):
    for dir_name in dir_names:
        yield from pathlib.Path(dir_name).rglob('eval_results.csv')

for results_file in iter_results_files('eval_orig_mini', 'eval_dsa'):
    model_dir = results_file.parent.parent
    attention_dir = model_dir.parent

    model = model_dir.stem
    attention = attention_dir.stem

    model_vals.append(model)
    attention_vals.append(attention)

    df = pd.read_csv(str(results_file), index_col=0)
    columns = df.columns
    overall_scores = df.iloc[-1]
    overall_scores_rows.append(overall_scores)

df_orig = pd.DataFrame(overall_scores_rows, columns=columns)
df_orig.insert(0, 'model', model_vals)
df_orig.insert(1, 'attention', attention_vals)
df_orig.reset_index(drop=True, inplace=True)
df_orig

In [ ]:
df = df_orig[
    (df_orig['model'] == '0035000') |
    (df_orig['model'] == '0025000') |
    (df_orig['model'] == '0055000') |
    (df_orig['model'] == '0050000') |
    (df_orig['model'] == '0045000') |
    (df_orig['model'] == '0040000')
]
df

In [ ]:
def plot_tracker_attention_comparison(
    df, x_col, y_col, x_label, y_label, x_units='\%', y_units='\%'
):
    def _build_axis_label(text, units=None):
        label = rf'$\textbf{{{text}}}$'
        if units is not None:
            label += f' [{units}]'
        return label
    
    def _get_attention_text(attention):
        return {'without_dsa': 'None', 'with_dsa': 'DSA'}[attention]
    
    fig, ax = plt.subplots(figsize=(16, 8), nrows=1, ncols=1)

    base_colors =  list(mcolors.BASE_COLORS.keys())
    unique_models = df['model'].unique().tolist()

    size = 30 ** 2
    for attention, group_attention_df in df.groupby(['attention']):
        marker = 'o' if 'without' in attention else 'D'
        
        for model, group_model_df in group_attention_df.groupby(['model']):
            xs, ys = group_model_df[x_col], group_model_df[y_col]    
            
            curr_color = base_colors[unique_models.index(model)]
            attention_text = _get_attention_text(attention)
            label = rf'${attention_text}, {str(int(model))}$'
            
            ax.scatter(
                xs, ys, s=size, c=curr_color, label=label, marker=marker,
                linewidth=2, alpha=0.7, antialiased=True
            )
    
    x_label_formatted = _build_axis_label(x_label, x_units)
    y_label_formatted = _build_axis_label(y_label, y_units)
    
    ax.set_title(
        r'$\textbf{Tracker Performance Comparison --- ' + 
        rf'Training Iterations and Deformable Siamese Attention --- {x_label} vs. {y_label}}}$'
    )
    ax.set_xlabel(x_label_formatted)
    ax.set_ylabel(y_label_formatted)
    ax.legend(loc='best', markerscale=0.5, fontsize=14)

    fig.tight_layout()

    return fig

fig_mota_motp = plot_tracker_attention_comparison(
    df, 'mota', 'motp', 'MOTA', 'MOTP'
)
fig_mota_motp.savefig('tracker_cmp_plot_mota_motp.png', dpi=300)

In [ ]:
fig_rec_prec = plot_tracker_attention_comparison(
    df, 'recall', 'precision', 'Recall', 'Precision'
)
fig_rec_prec.savefig('tracker_cmp_plot_rec_prec.png', dpi=300)